In [ ]:
import pandas as pd
import numpy as np
from transformers import GPT2Tokenizer

In [ ]:
pip install openai

In [ ]:
pip install torch


In [ ]:
pip install transformers

# Data Preprocessing

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/CloudSEK/Headline creation /train.csv')

In [ ]:
data.head()

,Headline,Category,Summary
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha..."
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to..."
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...


In [ ]:
data.shape

(170000, 3)

In [ ]:
train_data=data.dropna()
train_data.isnull().sum()

In [ ]:
train_data['text'] = train_data.apply(lambda row: f"Summary: {row['Summary']} Headline: {row['Headline']}", axis=1)

print(f"Number of rows after preprocessing: {len(train_data)}")


Number of rows after preprocessing: 150370


<ipython-input-16-35fd577dd506>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['text'] = train_data.apply(lambda row: f"Summary: {row['Summary']} Headline: {row['Headline']}", axis=1)


In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/CloudSEK/Headline creation /test.csv')

In [ ]:
test_data.head()

,Headline,Category,Summary
0,Biographer Elizabeth Winder on Sylvia Plath's ...,WELLNESS,"Ever since I read The Bell Jar, Sylvia Plath's..."
1,The Road Rage Warrior,WELLNESS,It's not easy driving in traffic with people c...
2,Deepak Chopra: Scientific Proof of God?,WELLNESS,"What it comes down to, then, and what science ..."
3,The Coava Kone Stainless Steel Coffee Filter: ...,FOOD & DRINK,"Once we thought about it for a minute, it star..."
4,Secrets That Women Need to Know When Buying Shoes,STYLE & BEAUTY,What's the biggest secret woman need to know a...


In [ ]:
test_data.isnull().sum()
test_data=test_data.dropna()

Headline     0
Category     0
Summary     83
dtype: int64

In [ ]:
test_data['text'] = test_data.apply(lambda row: f"Summary: {row['Summary']} Headline: {row['Headline']}", axis=1)
print(f"Number of rows after preprocessing: {len(test_data)}")

Number of rows after preprocessing: 39444


# Tokenization

In [ ]:
pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.

In [ ]:
from datasets import Dataset
from transformers import GPT2Tokenizer


In [ ]:
# Convert to Hugging Face dataset

dataset = Dataset.from_pandas(train_data[['text']])
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train']
val_dataset = dataset['test']

In [ ]:
# tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '<pad>'})

def tokenize_function(examples):

    encoding = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)
    encoding['labels'] = encoding['input_ids'].copy()
    return encoding

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask','labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask','labels'])

Map:   0%|          | 0/120296 [00:00<?, ? examples/s]

Map:   0%|          | 0/30074 [00:00<?, ? examples/s]

# Fine Tune GPT2 Model

In [ ]:
pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
pip show accelerate

Name: accelerate
Version: 0.31.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [ ]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments
import accelerate
import torch


# Load model
model = GPT2LMHeadModel.from_pretrained('gpt2')

model.resize_token_embeddings(len(tokenizer))

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


trainer.train()

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


# Result generation and model saving

In [ ]:
def generate_headline(summary):
    prompt = f"Summary: {summary} Headline:"
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs['input_ids'], max_length=50, num_beams=5, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


summary = "The stock market saw a significant increase today as major tech companies reported record earnings for the quarter."
headline = generate_headline(summary)
print("Generated Headline:", headline)

In [1]:

model.save_pretrained('./fine_tuned_gpt2')
tokenizer.save_pretrained('./fine_tuned_gpt2')

NameError: name 'model' is not defined

In [2]:
import torch

In [3]:
# Create pickle file
with open('fine_tuned_gpt2_model.pkl', 'wb') as model_file:
    torch.save(model.state_dict(), model_file)

with open('fine_tuned_gpt2_tokenizer.pkl', 'wb') as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)

print("Model and tokenizer have been saved as pickle files.")

NameError: name 'model' is not defined